In [1]:
import matplotlib.pyplot as plt
from  tensorflow import keras
import cv2
import numpy as np
import random
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,InputLayer,BatchNormalization , GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
from tensorflow.keras.models import load_model
from tqdm import tqdm
import tensorflow as tf


In [3]:
NUM_OF_CLASSES=10
IMG_SIZE=312

In [4]:
df_train_labels=pd.read_csv('labels.csv')
df_train_labels.sort_values(by='id',inplace=True)
Filtered_breeds=['beagle', 'chihuahua', 'doberman','french_bulldog', 'golden_retriever', 'malamute', 'pug',
                 'saint_bernard', 'scottish_deerhound','tibetan_mastiff']
df_train_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
def one_hot_encoding(index):
    encoded_arr=[]
    for i in range(0,NUM_OF_CLASSES):
        if i==index:
            encoded_arr.append(1)
        else:
            encoded_arr.append(0)
    return  encoded_arr

In [6]:
def read_data():
    x_train=[]
    y_train=[]
    i=0
    for image in sorted(os.listdir('train')):
        dog_breed=df_train_labels.iloc[i]['breed']
        img_title=df_train_labels.iloc[i]['id']
        if(dog_breed in Filtered_breeds):
            image_path = os.path.join('train',image)
            img=cv2.imread(image_path)
            img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
            x_train.append(np.array(img))
            dog_breed_index=Filtered_breeds.index(dog_breed)
            y_train.append(np.array(one_hot_encoding(dog_breed_index)))
        i=i+1
    return x_train,y_train

In [7]:
x_train,y_train = read_data()
print(len(x_train))
print(len(y_train))


841
841


In [13]:
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, y_train,test_size=0.2, random_state=42,shuffle=True)

X_train=np.array(X_train)
Y_train=np.array(Y_train)

X_validation=np.array(X_validation)
Y_validation=np.array(Y_validation)


In [14]:
MinCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    './model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [15]:
ResnetModel=ResNet50(weights='imagenet',include_top=False,input_shape=(312,312,3))

for layer in ResnetModel.layers[:-15]:
  layer.trainable=False

x=ResnetModel.layers[-1].output 
#x=keras.layers.Flatten()(x)
x=keras.layers.GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)  
x=Dropout(0.5)(x)

out=Dense(NUM_OF_CLASSES,activation='softmax')(x)

model=Model(ResnetModel.input,out)

op=keras.optimizers.Adam(lr=0.00002)

model.compile(loss='categorical_crossentropy',optimizer=op,metrics=['accuracy'])

aug=ImageDataGenerator(horizontal_flip=True)
model.fit_generator(generator=aug.flow(X_train,Y_train,batch_size=32),epochs=30,validation_data=(X_validation,Y_validation),verbose=True,callbacks=[MinCheckPoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
21/21 [==============================] - 9s 315ms/step - loss: 2.5908 - accuracy: 0.1414 - val_loss: 1.9187 - val_accuracy: 0.3314


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/30
21/21 [==============================] - 6s 279ms/step - loss: 1.8541 - accuracy: 0.3586 - val_loss: 1.4672 - val_accuracy: 0.5740
Epoch 3/30
21/21 [==============================] - 6s 279ms/step - loss: 1.4267 - accuracy: 0.5551 - val_loss: 1.0515 - val_accuracy: 0.7456
Epoch 4/30
21/21 [==============================] - 6s 282ms/step - loss: 1.0476 - accuracy: 0.7202 - val_loss: 0.7551 - val_accuracy: 0.8462
Epoch 5/30
21/21 [==============================] - 6s 282ms/step - loss: 0.7100 - accuracy: 0.8393 - val_loss: 0.5282 - val_accuracy: 0.9290
Epoch 6/30
21/21 [==============================] - 6s 277ms/step - loss: 0.5380 - accuracy: 0.9003 - val_loss: 0.4008 - val_accuracy: 0.9586
Epoch 7/30
21/21 [==============================] - 6s 290ms/step - loss: 0.3925 - accuracy: 0.9360 - val_loss: 0.3120 - val_accuracy: 0.9645
Epoch 8/30
21/21 [==============================] - 6s 273ms/step - loss: 0.3187 - accuracy: 0.9449 - val_loss: 0.2668 - val_accuracy: 0.9527
Epoch 

In [40]:
from sklearn.metrics import classification_report , confusion_matrix , roc_auc_score

In [24]:
loss,accuracy=model.evaluate(X_validation,Y_validation)
print('The Final Accuracy is {}'.format(accuracy))

6/6 [==============================] - 1s 150ms/step - loss: 0.1494 - accuracy: 0.9586
The Final Accuracy is 0.9585798978805542


In [38]:
y_pred=model.predict(X_validation)
y_pred_bool=np.argmax(y_pred,axis=1)
y_test_bool=np.argmax(Y_validation,axis=1)
print(classification_report(y_test_bool,y_pred_bool,target_names=Filtered_breeds))


                    precision    recall  f1-score   support

            beagle       1.00      1.00      1.00        21
         chihuahua       0.87      1.00      0.93        13
          doberman       1.00      1.00      1.00        16
    french_bulldog       0.83      0.91      0.87        11
  golden_retriever       1.00      0.91      0.95        22
          malamute       0.93      0.88      0.90        16
               pug       0.93      0.87      0.90        15
     saint_bernard       1.00      1.00      1.00        18
scottish_deerhound       1.00      1.00      1.00        24
   tibetan_mastiff       0.93      1.00      0.96        13

          accuracy                           0.96       169
         macro avg       0.95      0.96      0.95       169
      weighted avg       0.96      0.96      0.96       169



In [39]:
print(confusion_matrix(y_test_bool,y_pred_bool))

[[21  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  1  0  0  0]
 [ 0  1  0  0 20  1  0  0  0  0]
 [ 0  1  0  0  0 14  0  0  0  1]
 [ 0  0  0  2  0  0 13  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  0  0  0  0  0  0  0 24  0]
 [ 0  0  0  0  0  0  0  0  0 13]]


In [42]:
print(roc_auc_score(Y_validation,y_pred))

0.9987591361334642
